In [11]:
import sys
import os
import pandas as pd
import numpy as np

"""
If data is not ready, we need download the raw data and unzip the files in the same route
If data is ready, return the DataFrame
"""
def if_data_ready(filename):
    if os.path.exists(filename):
        print (filename+" already exist")
        return pd.read_csv(filename)
    else:
        print (filename+" does not exist, please check the file")
        sys.exit()

def is_new_trip(position,trip_id):
    index = 0
    trip = [0]*len(position['Date'])
    trip [0] = trip_id
    while index < len(position['Date'])-1:
        index += 1
        if index ==len(position['Date'])-1:
            if (position.iat[index,8] - position.iat[index-1,8] == 8870) or (position.iat[index,8] - position.iat[index-1,8]<100 and position.iat[index,3] == position.iat[index-1,3] and position.iat[index,4] == position.iat[index-1,4]):
                num_data = trip.count(trip_id)
                if trip.count(trip_id) < 5:
                    for i in range(index-1, index-num_data-1, -1):
                        trip[i]=0
                    continue
                else:
                    trip[index] = trip_id
                    trip_id += 1
                    continue        
        if (position.iat[index,8] - position.iat[index-1,8] == 8870) or (position.iat[index,8] - position.iat[index-1,8]<100 and position.iat[index,3] == position.iat[index-1,3] and position.iat[index,4] == position.iat[index-1,4]):
            trip[index] = trip_id
        else:
            ##delete trips that is too short
            num_data = trip.count(trip_id)
            if trip.count(trip_id) < 5:
                for i in range(index-1, index-num_data-1, -1):
                    trip[i]=0  ## the data point would be marked 0
            else:
                trip_id += 1
                trip[index] = trip_id
    position.insert(9, "Trip", trip)
    return trip_id
def split_trip():
    #shipname = pd.value_counts(raw_data["ShipName"]).head(10).keys()
    shipname = list(pd.value_counts(raw_data["ShipName"]).keys()) #949 shipnames
    shipname = [x for x in shipname if 'SAN CARLOS' not in x]
    shipname = [x for x in shipname if 'LA SUERTE' not in x] 
    shipname = [x for x in shipname if 'LABORDE' not in x]
    shipname = [x for x in shipname if 'CONOCIDO-26' not in x]
    shipname = [x for x in shipname if 'NaSa DEL ROSARIO,STMA TRINIDAD' not in x]
    shipname = [x for x in shipname if 'Vrouwe Margaretha Maria' not in x]
    shipname = [x for x in shipname if 'EL MARLLORQUÍN' not in x] 
    shipname = [x for x in shipname if 'LA ORBE' not in x]
    shipname = [x for x in shipname if 'LA JUNO' not in x]
    shipname = [x for x in shipname if 'DESCONOCIDO-01' not in x]
    shipname = [x for x in shipname if 'DESCONOCIDO-08' not in x]
    shipname = [x for x in shipname if 'DESCONOCIDO-21' not in x]
    shipname = [x for x in shipname if 'DESCONOCIDO-11' not in x]   #Some ships could cause error and need to be removed manually
    position_all = []
    trip_id = 1
    for ship in shipname:
        position = raw_data[raw_data["ShipName"]==ship][pd.notnull(raw_data["Lat3"])][pd.notnull(raw_data["Lon3"])][["Lat3","Lon3","VoyageFrom","VoyageTo","Year","Month","Day","UTC","Nationality","RecID"]]
        position = position.sort_values(by='UTC', ascending=True)
        position.insert(8, "Date", position['UTC'])
        position['Date'] = position['Date'] // 100
        #print(ship, end=" ")   ##For testing which ship got errer message
        trip_id = is_new_trip(position,trip_id)
        position_each_ship = list(zip([ship]*len(position["Lat3"]),position["Nationality"],position["VoyageFrom"],position["VoyageTo"],position["Year"],position["Date"],position["Lat3"],position["Lon3"],position["RecID"],position["Trip"]))
        position_all += position_each_ship
    return position_all
if __name__ == "__main__":
    file_path ="C:\\Users\\Administrator\\data\\data\\CLIWOC15.csv"
    raw_data = if_data_ready(file_path) ##just read the data frame
    df = pd.DataFrame(split_trip(), columns=['ShipName',"Nationality","VoyageFrom","VoyageTo","Year","Date",'Latitude', 'Longtitude', "RecID",'Trip'])
#    df.to_excel('Trip_Data_new.xlsx',index=False)


            

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3006: DtypeWarning: Columns (5,6,7,8,11,13,18,19,23,24,25,26,28,29,30,34,35,38,43,44,46,73,77,81,82,84,85,87,88,94,96,97,98,99,111,114,116,119,120,122,124,125,127,129,131,133,135,137,140) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
C:\Anaconda3\lib\site-packages\pandas\core\frame.py:1997: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


C:\Users\Administrator\data\data\CLIWOC15.csv already exist


In [ ]:
#raw_data.insert(141, "Trip_ID",0)
#trip_id = [0]*len(raw_data["Trip_ID"])
trip_id = [0]*len(raw_data["RecID"]) #Set a new column for trip_id
for i in range(len(raw_data["RecID"])):
    try: #if raw_data's RecID macth RecID in df, then return the trip_id for that line
        trip_id[i]=df.loc[df[df["RecID"]==raw_data.iat[i,0]]["Trip"].keys()[0],"Trip"] 
    except:
        pass

#print(trip_id[-50:])



In [35]:
print(df[df["RecID"]==548]["Trip"])   #Test for datatype of this sentence, not necessary for main function

199168    3470
Name: Trip, dtype: int64


In [ ]:
raw_data.insert(141, "Trip_ID", trip) ##add the trip id colunmn to the end of raw data